In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [2]:
df = pd.read_csv(Path('./data/20210626.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice]
0,22/06/2021 17:18:09,A Fantastic Woman (2017),Taste of Cherry (1997),Hyenas (1992),ANIMALS,ANIMAL CO-STAR,[Default to Nicolas Cage],"ACTUALLY, BASEBALL IS THE GREATEST MOVIE SPORT"
1,22/06/2021 17:21:38,Taste of Cherry (1997),A Fantastic Woman (2017),Hyenas (1992),"ACTUALLY, BASEBALL IS THE GREATEST MOVIE SPORT",ANIMAL CO-STAR,ANIMALS,[Default to Nicolas Cage]
2,22/06/2021 17:28:08,Hyenas (1992),Taste of Cherry (1997),A Fantastic Woman (2017),ANIMAL CO-STAR,ANIMALS,[Default to Nicolas Cage],"ACTUALLY, BASEBALL IS THE GREATEST MOVIE SPORT"
3,23/06/2021 08:47:33,Hyenas (1992),A Fantastic Woman (2017),Taste of Cherry (1997),ANIMAL CO-STAR,"ACTUALLY, BASEBALL IS THE GREATEST MOVIE SPORT",[Default to Nicolas Cage],ANIMALS
4,24/06/2021 10:29:19,Hyenas (1992),Taste of Cherry (1997),A Fantastic Woman (2017),ANIMAL CO-STAR,"ACTUALLY, BASEBALL IS THE GREATEST MOVIE SPORT",ANIMALS,[Default to Nicolas Cage]


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['A Fantastic Woman (2017)', 'Taste of Cherry (1997)',
       'Hyenas (1992)'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'A Fantastic Woman (2017)': <Candidate('A Fantastic Woman (2017)')>,
 'Taste of Cherry (1997)': <Candidate('Taste of Cherry (1997)')>,
 'Hyenas (1992)': <Candidate('Hyenas (1992)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(A Fantastic Woman (2017), Taste of Cherry (1997), Hyenas (1992))>,
 <Ballot(Taste of Cherry (1997), A Fantastic Woman (2017), Hyenas (1992))>,
 <Ballot(Hyenas (1992), Taste of Cherry (1997), A Fantastic Woman (2017))>,
 <Ballot(Hyenas (1992), A Fantastic Woman (2017), Taste of Cherry (1997))>,
 <Ballot(Hyenas (1992), Taste of Cherry (1997), A Fantastic Woman (2017))>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

FINAL RESULT
Candidate                   Votes  Status
------------------------  -------  --------
Hyenas (1992)                   3  Elected
Taste of Cherry (1997)          1  Rejected
A Fantastic Woman (2017)        1  Rejected



In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

FINAL RESULT
Candidate                                         Votes  Status
----------------------------------------------  -------  --------
ANIMAL CO-STAR                                        3  Elected
ACTUALLY, BASEBALL IS THE GREATEST MOVIE SPORT        1  Rejected
ANIMALS                                               1  Rejected
[Default to Nicolas Cage]                             0  Rejected

